In [1]:
%cd MaxText

/home/lizhiyu/maxtext/MaxText


/home/lizhiyu/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
!ls /home/lizhiyu/maxtext/assets/tokenizer.mistral-v1

/home/lizhiyu/maxtext/assets/tokenizer.mistral-v1


In [3]:
import pyconfig
import os


argv = [
    'MaxText/train.py', '/home/lizhiyu/maxtext/MaxText/configs/base.yml',  # base arg
    "run_name=mixtral-mlperf",
    "model_name=mixtral-8x22b",
    "steps=10",
    "per_device_batch_size=6",
    "enable_checkpointing=false", 
    "async_checkpointing=false",
    "remat_policy=full",
    "dataset_path=gs://mlperf-llm-public2",
    "dataset_name=c4/en:3.0.4", 
    "eval_dataset_name=c4/en:3.0.4",  # c4/en:3.0.4 is the untokenized eval dataset
    "max_target_length=32768", 
    "base_output_directory=/tmp/",
    "dataset_type=c4_mlperf",
    "tokenizer_path=/home/lizhiyu/maxtext/assets/tokenizer.mistral-v1",
    "attention=flash",
    "eval_interval=1",
    ]
pyconfig.initialize(argv)
cfg = pyconfig.config
config = cfg

2025-01-10 08:34:52.433167: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736498092.448828 1100032 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736498092.453477 1100032 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Updating keys from env and command line: ['run_name', 'model_name', 'enable_checkpointing', 'async_checkpointing', 'remat_policy', 'attention', 'base_output_directory', 'tokenizer_path', 'per_device_batch_size', 'dataset_type', 'dataset_path', 'dataset_name', 'eval_dataset_name', 'steps', 'max_target_length', 'eval_interval']
Running Model: mixtral-8x22b
Updating following parameters in config

base_emb_dim: 6144
base_num_query_heads: 48
base_num_kv_heads: 8
base_mlp_dim: 16384
base_num_decoder_layers: 56
head_dim: 128
mlp_activations: ['silu', 'linear']
vocab_size: 32768
enable_dropout: False
logits_via_embedding: False
normalization_layer_epsilon: 1e-05
num_experts: 8
num_experts_per_tok: 2
rope_max_timescale: 1000000
decoder_block: mistral
Updating keys from model: ['base_emb_dim', 'base_num_query_heads', 'base_num_kv_heads', 'base_mlp_dim', 'base_num_decoder_layers', 'head_dim', 'mlp_activations', 'vocab_size', 'enable_dropout', 'logits_via_embedding', 'normalization_layer_epsilon'

In [4]:
%env TFDS_DATA_DIR={cfg.dataset_path}

env: TFDS_DATA_DIR=gs://mlperf-llm-public2


In [5]:
import numpy as np
from input_pipeline.input_pipeline_interface import create_data_iterator
from max_utils import create_device_mesh
from jax.sharding import Mesh
import jax.numpy as jnp

devices_array = create_device_mesh(cfg)
mesh = Mesh(devices_array, cfg.mesh_axes)

train_iter, eval_iter = create_data_iterator(config, mesh)

total_weights = 0
for i, batch in enumerate(eval_iter):
    total_weights += jnp.sum(batch["targets_segmentation"] != 0)

print(f"{total_weights=}")
                    

/home/lizhiyu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Num_devices: 4, shape (1, 1, 4, 1, 1, 1, 1, 1, 1)


2025-01-10 08:34:58.250825: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Tokenizer path: /home/lizhiyu/maxtext/assets/tokenizer.mistral-v1
Tokenizer path: /home/lizhiyu/maxtext/assets/tokenizer.mistral-v1


normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Tokenizer path: /home/lizhiyu/maxtext/assets/tokenizer.mistral-v1
Tokenizer path: /home/lizhiyu/maxtext/assets/tokenizer.mistral-v1
Eval data has 388 local entries, padding now with 20 extra entries to get 17 batches.
total_weights=Array(12694503, dtype=int32)


In [6]:
# reference: https://github.com/suexu1025/MoE_study/blob/main/clm/clm_datasets.py#L250-L265
# 'eval/num_tokens': 12694891: total number of tokens in eval dataset
# 'eval/total_weights': 12694503: total number of next tokens prediction in eval dataset
assert total_weights == 12694503